Initial Part Constant For All Models

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf
from tensorflow import keras

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Specify the path to your CSV file
csv_file_path = '/content/drive/MyDrive/IS_STLF_Dataset_2012_2015/ERCOT_2012_to_2015.csv'

Model 2: Finding best hyperparameters/ architecture

In [ ]:
# Load the dataset into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
df.head(5)

,Date,Day of Week(0-6),Day of the Month(1-31),Month of the Year(1-12),Time of Day(0-23),Holidays,Humidity,Temperature,Load WEST
0,1/1/2012,6,1,1,1,1,34.89,13.2,849.000892
1,1/1/2012,6,1,1,2,1,37.89,12.0,845.097364
2,1/1/2012,6,1,1,3,1,46.46,10.2,840.902849
3,1/1/2012,6,1,1,4,1,54.48,8.4,845.452257
4,1/1/2012,6,1,1,5,1,63.22,6.6,862.369386


In [ ]:
# Extract features and target variable
features = ['Day of Week(0-6)', 'Day of the Month(1-31)', 'Month of the Year(1-12)', 'Time of Day(0-23)', 'Holidays', 'Humidity', 'Temperature']
target_variable = 'Load WEST'
data = df[['Day of Week(0-6)', 'Day of the Month(1-31)', 'Month of the Year(1-12)', 'Time of Day(0-23)', 'Holidays', 'Humidity', 'Temperature', 'Load WEST']]

In [ ]:
# Normalize data using Min-Max normalization
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

In [ ]:
type(data_normalized)

numpy.ndarray

In [ ]:
data_normalized.shape

(35064, 8)

In [ ]:
#2015 starts fromrow 26304
#checking
data.iloc[26303]

Day of Week(0-6)              3.00000
Day of the Month(1-31)        1.00000
Month of the Year(1-12)       1.00000
Time of Day(0-23)             0.00000
Holidays                      1.00000
Humidity                     77.15000
Temperature                   1.20000
Load WEST                  1504.43762
Name: 26303, dtype: float64

In [ ]:
data_normalized[26303]

array([0.5       , 0.        , 0.        , 0.        , 1.        ,
       0.75412989, 0.20517928, 0.69694446])

In [ ]:
train_set=data_normalized[:26303]
test_set=data_normalized[26303:]

In [ ]:
print(train_set.shape)
print(test_set.shape)

(26303, 8)
(8761, 8)


In [ ]:
# Function to prepare sequences for LSTM
def prepare_sequences(data, look_back):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), :])
        y.append(data[i + look_back, -1])  # Assuming Load WEST is the last column
    return np.array(X), np.array(y)

# Set the look-back window
look_back = 12 # Adjust this value based on your analysis #14 turned out to be best

In [ ]:
# Prepare sequences for train set
X_train_I, y_train_I = prepare_sequences(train_set, look_back)

In [ ]:
# Prepare sequences for test set
X_test, y_test = prepare_sequences(test_set, look_back)

In [ ]:
print(X_train_I.shape)
print(y_train_I.shape)
print(X_test.shape)
print(y_test.shape)

(26291, 12, 8)
(26291,)
(8749, 12, 8)
(8749,)


In [ ]:
# Split the data into training, validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_I, y_train_I, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(21032, 12, 8)
(21032,)
(5259, 12, 8)
(5259,)


In [ ]:
X_train.shape[1]

12

In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming you have X_train, y_train, X_val, y_val defined

# Function to build the model
def build_model(hp):
    model = Sequential()

    # Tune the number of LSTM units
    model.add(LSTM(units=hp.Int('units', min_value=30, max_value=80, step=10),
                   return_sequences=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))

    # Tune the number of intermediate LSTM layers
    for _ in range(hp.Int('num_layers', min_value=1, max_value=5)):
        model.add(LSTM(units=hp.Int('units', min_value=30, max_value=80, step=10), return_sequences=True))

    # Manually set return_sequences=False for the last LSTM layer
    model.add(LSTM(units=hp.Int('units_last', min_value=30, max_value=80, step=10), return_sequences=False))

    # Dense layer for final prediction
    model.add(Dense(units=1))

    # Choose between Adam and Nadam optimizers
    optimizer_choice = hp.Choice('optimizer', ['adam', 'nadam'])

    # Compile the model
    model.compile(optimizer=optimizer_choice, loss='mean_squared_error')

    return model

# Define the tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Adjust as needed
    executions_per_trial=1,
    directory='my_tuner_directory',
    project_name='lstm_tuning'
)

# Early stopping to stop training when the validation loss doesn't improve
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Perform the search
tuner.search(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
final_model = build_model(best_hps)


Trial 10 Complete [00h 08m 13s]
val_loss: 0.00014028121950104833

Best val_loss So Far: 0.00010825267963809893
Total elapsed time: 01h 28m 37s


In [ ]:
best_hps

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:")
print(f"Number of Units: {best_hps.get('units')}")
print(f"Number of Intermediate Layers: {best_hps.get('num_layers')}")
print(f"Number of Units in Last Layer: {best_hps.get('units_last')}")

Best Hyperparameters:
Number of Units: 40
Number of Intermediate Layers: 2
Number of Units in Last Layer: 80


In [ ]:
type(best_hps)

keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters

In [ ]:
# Assume you have obtained best_hps as mentioned before

# Print all attributes and their values
for key, value in best_hps.__dict__.items():
    print(f"{key}: {value}")


_name_scopes: []
_conditions: []
_hps: defaultdict(<class 'list'>, {'units': [Int(name: 'units', min_value: 30, max_value: 80, step: 10, sampling: linear, default: 30)], 'num_layers': [Int(name: 'num_layers', min_value: 1, max_value: 5, step: 1, sampling: linear, default: 1)], 'units_last': [Int(name: 'units_last', min_value: 30, max_value: 80, step: 10, sampling: linear, default: 30)], 'optimizer': [Choice(name: 'optimizer', values: ['adam', 'nadam'], ordered: False, default: adam)]})
_space: [Int(name: 'units', min_value: 30, max_value: 80, step: 10, sampling: linear, default: 30), Int(name: 'num_layers', min_value: 1, max_value: 5, step: 1, sampling: linear, default: 1), Int(name: 'units_last', min_value: 30, max_value: 80, step: 10, sampling: linear, default: 30), Choice(name: 'optimizer', values: ['adam', 'nadam'], ordered: False, default: adam)]
values: {'units': 40, 'num_layers': 2, 'units_last': 80, 'optimizer': 'nadam'}
active_scopes: []
inactive_scopes: []


In [ ]:
# Train the final model
history = final_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate on the test set
test_loss = final_model.evaluate(X_test, y_test)

# You can access the best model using the 'best_model' variable

Epoch 1/100
658/658 [==============================] - 34s 36ms/step - loss: 0.0088 - val_loss: 0.0027
Epoch 2/100
658/658 [==============================] - 24s 36ms/step - loss: 0.0016 - val_loss: 8.8388e-04
Epoch 3/100
658/658 [==============================] - 22s 33ms/step - loss: 7.6310e-04 - val_loss: 0.0017
Epoch 4/100
658/658 [==============================] - 23s 35ms/step - loss: 4.7016e-04 - val_loss: 3.0672e-04
Epoch 5/100
658/658 [==============================] - 24s 37ms/step - loss: 3.0676e-04 - val_loss: 2.2553e-04
Epoch 6/100
658/658 [==============================] - 21s 33ms/step - loss: 2.6332e-04 - val_loss: 5.1869e-04
Epoch 7/100
658/658 [==============================] - 21s 32ms/step - loss: 2.4859e-04 - val_loss: 2.9260e-04
Epoch 8/100
658/658 [==============================] - 22s 33ms/step - loss: 2.2422e-04 - val_loss: 3.0151e-04
Epoch 9/100
658/658 [==============================] - 23s 34ms/step - loss: 2.0602e-04 - val_loss: 2.4478e-04
Epoch 10/100
658/

In [ ]:
test_loss

9.704315743874758e-05

In [ ]:
X_test.shape

(8749, 12, 8)

In [ ]:
X_test[:, -1, :].shape

(8749, 8)

In [ ]:
# Make predictions
predictions = final_model.predict(X_test)

274/274 [==============================] - 4s 8ms/step


In [ ]:
predictions.shape

(8749, 1)

In [ ]:
y_test.shape

(8749,)

In [ ]:
# Invert the predictions and actual values to compare with the original scale
predictions_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], predictions), axis=1))[:, -1]
y_test_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test_original, predictions_original)
r2 = r2_score(y_test_original, predictions_original)

print(f'Mean Absolute Percentage Error (MAPE): {mape * 100:.2f}%')
print(f'R-squared (R2): {r2:.4f}')

Mean Absolute Percentage Error (MAPE): 0.83%
R-squared (R2): 0.9972


In [ ]:
# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test_original, predictions_original)
rmse = np.sqrt(mean_squared_error(y_test_original, predictions_original))

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

Mean Absolute Error (MAE): 9.066112954820625
Root Mean Squared Error (RMSE): 12.33435306942593


In [ ]:
# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test_original, predictions_original)
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 152.13626564125684


In [ ]:
final_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 12, 40)            7840      
                                                                 
 lstm_5 (LSTM)               (None, 12, 40)            12960     
                                                                 
 lstm_6 (LSTM)               (None, 12, 40)            12960     
                                                                 
 lstm_7 (LSTM)               (None, 80)                38720     
                                                                 
 dense_1 (Dense)             (None, 1)                 81        
                                                                 
Total params: 72561 (283.44 KB)
Trainable params: 72561 (283.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
for i, layer in enumerate(final_model.layers):
    print(f"Layer {i+1}: {layer.name} - Type: {type(layer).__name__}")
    if hasattr(layer, 'units'):
        print(f"Number of Neurons: {layer.units}")
    print("-----------------------------------")


Layer 1: lstm_4 - Type: LSTM
Number of Neurons: 40
-----------------------------------
Layer 2: lstm_5 - Type: LSTM
Number of Neurons: 40
-----------------------------------
Layer 3: lstm_6 - Type: LSTM
Number of Neurons: 40
-----------------------------------
Layer 4: lstm_7 - Type: LSTM
Number of Neurons: 80
-----------------------------------
Layer 5: dense_1 - Type: Dense
Number of Neurons: 1
-----------------------------------


Model 3: Using Best Hyperparameters to obtain optimum look-back window

In [ ]:
# Load the dataset into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
df.head(5)

,Date,Day of Week(0-6),Day of the Month(1-31),Month of the Year(1-12),Time of Day(0-23),Holidays,Humidity,Temperature,Load WEST
0,1/1/2012,6,1,1,1,1,34.89,13.2,849.000892
1,1/1/2012,6,1,1,2,1,37.89,12.0,845.097364
2,1/1/2012,6,1,1,3,1,46.46,10.2,840.902849
3,1/1/2012,6,1,1,4,1,54.48,8.4,845.452257
4,1/1/2012,6,1,1,5,1,63.22,6.6,862.369386


In [ ]:
# Extract features and target variable
features = ['Day of Week(0-6)', 'Day of the Month(1-31)', 'Month of the Year(1-12)', 'Time of Day(0-23)', 'Holidays', 'Humidity', 'Temperature']
target_variable = 'Load WEST'
data = df[['Day of Week(0-6)', 'Day of the Month(1-31)', 'Month of the Year(1-12)', 'Time of Day(0-23)', 'Holidays', 'Humidity', 'Temperature', 'Load WEST']]

In [ ]:
# Normalize data using Min-Max normalization
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

In [ ]:
type(data_normalized)

numpy.ndarray

In [ ]:
data_normalized.shape

(35064, 8)

In [ ]:
#2015 starts from row 26304
#checking
data.iloc[26303]

Day of Week(0-6)              3.00000
Day of the Month(1-31)        1.00000
Month of the Year(1-12)       1.00000
Time of Day(0-23)             0.00000
Holidays                      1.00000
Humidity                     77.15000
Temperature                   1.20000
Load WEST                  1504.43762
Name: 26303, dtype: float64

In [ ]:
data_normalized[26303]

array([0.5       , 0.        , 0.        , 0.        , 1.        ,
       0.75412989, 0.20517928, 0.69694446])

In [ ]:
train_set=data_normalized[:26303]
test_set=data_normalized[26303:]

In [ ]:
print(train_set.shape)
print(test_set.shape)

(26303, 8)
(8761, 8)


In [ ]:
# Function to prepare sequences for LSTM
def prepare_sequences(data, look_back):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), :])
        y.append(data[i + look_back, -1])  # Assuming Load WEST is the last column
    return np.array(X), np.array(y)

# Set the look-back window
# look_back = 12 # Adjust this value based on your analysis

In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
# Best LSTM Model Architecture found after hyperparameter tuning
'''from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the model
model = Sequential()

# Add LSTM layers
model.add(LSTM(units=40, return_sequences=True, input_shape=(12, features_count)))
model.add(LSTM(units=40, return_sequences=True))
model.add(LSTM(units=40, return_sequences=True))
model.add(LSTM(units=80))

# Add Dense layer for final prediction
model.add(Dense(units=1))

# Print model summary
model.summary()
'''

In [ ]:
# Best LSTM Model Architecture
'''
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Function to build the model with specific hyperparameters
def build_model_with_params(units, num_layers, units_last, input_shape):
    model = Sequential()

    # Add the first LSTM layer
    model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))

    # Add intermediate LSTM layers
    for _ in range(num_layers):
        model.add(LSTM(units=units, return_sequences=True))

    # Add the last LSTM layer with return_sequences=False
    model.add(LSTM(units=units_last, return_sequences=False))

    # Add a Dense layer for final prediction
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='nadam', loss='mean_squared_error')

    return model

# Best Hyperparameters obtained from tuning
best_units = 40
best_num_layers = 2
best_units_last = 80

# Input shape based on your data
input_shape = (12, 8)  # Update features_count based on your data

# Build the model with best hyperparameters
best_model = build_model_with_params(units=best_units, num_layers=best_num_layers, units_last=best_units_last, input_shape=input_shape)

# Print model summary
best_model.summary()

'''

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 12, 40)            7840      
                                                                 
 lstm_1 (LSTM)               (None, 12, 40)            12960     
                                                                 
 lstm_2 (LSTM)               (None, 12, 40)            12960     
                                                                 
 lstm_3 (LSTM)               (None, 80)                38720     
                                                                 
 dense (Dense)               (None, 1)                 81        
                                                                 
Total params: 72561 (283.44 KB)
Trainable params: 72561 (283.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


# Function to build the model with specific hyperparameters
def build_model_with_params(units, num_layers, units_last, input_shape):
    model = Sequential()

    # Add the first LSTM layer
    model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))

    # Add intermediate LSTM layers
    for _ in range(num_layers):
        model.add(LSTM(units=units, return_sequences=True))

    # Add the last LSTM layer with return_sequences=False
    model.add(LSTM(units=units_last, return_sequences=False))

    # Add a Dense layer for final prediction
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='nadam', loss='mean_squared_error')

    return model

# Best Hyperparameters obtained from tuning
best_units = 40
best_num_layers = 2
best_units_last = 80

# Input shape based on your data
# input_shape = (X_train.shape[1], X_train.shape[2])

# Automate for different look-back windows
look_back_values = [2, 4 ,6, 8 ,10 ,12 ,14 ,16 ,18 ,20 , 22, 24]

for look_back in look_back_values:
    #X, y = prepare_sequences(data_normalized, look_back)

    # Split the data into training, validation, and testing sets
    #X_train_current, X_temp, y_train_current, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    #X_val_current, X_test, y_val_current, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Prepare sequences for train set
    X_train_I, y_train_I = prepare_sequences(train_set, look_back)
    # Prepare sequences for test set
    X_test, y_test = prepare_sequences(test_set, look_back)

    # Split the data into training, validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_I, y_train_I, test_size=0.2, random_state=42)

    # Build the model with specific hyperparameters
    current_model = build_model_with_params(units=best_units, num_layers=best_num_layers, units_last=best_units_last, input_shape=(X_train.shape[1], X_train.shape[2]))

    # Prepare sequences with the current look_back
   # X_train_current, y_train_current = prepare_sequences(X_train, look_back)
   # X_val_current, y_val_current = prepare_sequences(X_val, look_back)

    # Train the model with EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    current_model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Make predictions on the validation set
    y_test_pred = current_model.predict(X_test)

    # Invert the predictions and actual values to compare with the original scale
    predictions_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test_pred.reshape(-1, 1)), axis=1))[:, -1]
    y_test_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

    # Calculate and print metrics
    mse = mean_squared_error(y_test_original, predictions_original)
    mae = mean_absolute_error(y_test_original, predictions_original)
    mape = mean_absolute_percentage_error(y_test_original, predictions_original)*100
    rmse = np.sqrt(mse)
    #r2 = r2_score(y_test_original, predictions_original)

    print(f"Look Back: {look_back}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"RMSE: {rmse}")
    #print(f"R2: {r2}")
    print("--------------------")




Epoch 1/100
658/658 [==============================] - 33s 19ms/step - loss: 0.0097 - val_loss: 0.0029
Epoch 2/100
658/658 [==============================] - 7s 11ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 3/100
658/658 [==============================] - 10s 15ms/step - loss: 5.9025e-04 - val_loss: 5.9580e-04
Epoch 4/100
658/658 [==============================] - 7s 11ms/step - loss: 4.0489e-04 - val_loss: 3.7866e-04
Epoch 5/100
658/658 [==============================] - 10s 14ms/step - loss: 3.6192e-04 - val_loss: 3.3711e-04
Epoch 6/100
658/658 [==============================] - 7s 11ms/step - loss: 3.5040e-04 - val_loss: 3.1414e-04
Epoch 7/100
658/658 [==============================] - 10s 15ms/step - loss: 3.4579e-04 - val_loss: 3.1322e-04
Epoch 8/100
658/658 [==============================] - 7s 11ms/step - loss: 3.3346e-04 - val_loss: 3.3253e-04
Epoch 9/100
658/658 [==============================] - 9s 14ms/step - loss: 3.3299e-04 - val_loss: 3.7841e-04
Epoch 10/100
658/658 [